## Before training

This program saves the last 3 generations of models to Google Drive. Since 1 generation of models is >1GB, you should have at least 3GB of free space in Google Drive. If you do not have such free space, it is recommended to create another Google Account.

Training requires >10GB VRAM. (T4 should be enough) Inference does not require such a lot of VRAM.

## Installation

In [ ]:
#@title Connect to colab runtime and check GPU
!nvidia-smi

In [ ]:
#@title Install dependencies
#@markdown pip may fail to resolve dependencies and raise ERROR, but it can be ignored.
!python -m pip install -U pip setuptools wheel
%pip install -U ipython~=7.34.0

#@markdown Branch (for development)
BRANCH = "none" #@param {"type": "string"}
if BRANCH == "none":
    %pip install -U so-vits-svc-fork
else:
    %pip install -U git+https://github.com/34j/so-vits-svc-fork.git@{BRANCH}

In [ ]:
#@title Install rclone for OneDrive
# !apt install -y fuse3
!sudo -v ; curl https://rclone.org/install.sh | sudo bash
!rclone -V

In [ ]:
#@markdown ### After the execution is completed, the runtime will **automatically restart**
# exit()

---

## **Restart runtime**

After running the cell above, you'll need to restart the Colab runtime because we installed a different version of numpy.

`Runtime -> Restart runtime`

---

## Settings

In [ ]:
RCLONE_CONFIG = "drive/MyDrive/rclone.conf" #@param {type: "string"}
CHARACTER = "kiritan" #@param {type: "string"}
DO_EXTRACT_VOCALS = False
CONFIG_FILE = "configs/44k/config.json" #@param {type: "string"}
MODEL_PATH = "drive/MyDrive/so-vits-svc-fork/logs/44k/" #@param {type: "string"}
MODEL_REPO_ID = "issenn/kiritan"
KEEP_CKPTS = 10 #@param {type: "number"}
CKPT_NAME_BY_STEP = True #@param {type: "boolean"}
CONFIG_TYPE = "so-vits-svc-4.0v1" #@param ["quickvc", "so-vits-svc-4.0v1-legacy", "so-vits-svc-4.0v1"]

## Data Preparation

In [ ]:
#@title Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title Mount OneDrive
# ![ ! -d onedrive ] && mkdir onedrive
# !rclone mount colab:/ /content/onedrive --config {RCLONE_CONFIG} --vfs-cache-mode full --daemon

In [ ]:
"""
Here, we override config to have num_workers=0 because
of a limitation in HF Spaces Docker /dev/shm.
"""

import json
from pathlib import Path
import multiprocessing

def update_config(config_file=CONFIG_FILE):
    config_path = Path(config_file)
    data = json.loads(config_path.read_text())
    # data['train']['eval_interval'] = 200
    # data['train']['epochs'] = 10000
    data['train']['learning_rate'] = 0.00005
    # data['train']['batch_size'] = 32
    data['train']['keep_ckpts'] = KEEP_CKPTS
    data['train']['num_workers'] = multiprocessing.cpu_count()
    data['train']['ckpt_name_by_step'] = CKPT_NAME_BY_STEP
    # data['train']['persistent_workers'] = True
    # data['train']['push_to_hub'] = True
    # data['train']['repo_id'] = MODEL_REPO_ID # tuple(data['spk'])[0]
    # data['train']['private'] = True
    config_path.write_text(json.dumps(data, indent=2, sort_keys=False))

## Run all Preprocessing Steps

In [ ]:
#@title Clean dataset directory
!rm -r "dataset_raw_raw"
!rm -r "dataset_raw"
!rm -r "dataset"

In [ ]:
#@title Make dataset directory
!mkdir -p "dataset_raw_raw"
!mkdir -p "dataset_raw"
!mkdir -p "dataset"

In [ ]:
#@title Copy your dataset_raw_raw
#@markdown **We assume that your dataset_raw_raw is in your Google Drive's `so-vits-svc-fork/dataset_raw_raw/(speaker_name)` directory.**
DATASET_NAME = CHARACTER
!cp -R /content/drive/MyDrive/so-vits-svc-fork/dataset_raw_raw/{DATASET_NAME}/ -t "dataset_raw_raw/"

In [ ]:
#@title Automatic split audio files into multiple files
!svc pre-split

In [ ]:
#@title Copy your dataset_raw
#@markdown **We assume that your dataset_raw is in your Google Drive's `so-vits-svc-fork/dataset_raw/(speaker_name)` directory.**
DATASET_NAME = CHARACTER
!cp -R /content/drive/MyDrive/so-vits-svc-fork/dataset_raw/{DATASET_NAME}/ -t "dataset_raw/"

In [ ]:
#@title Download dataset (Tsukuyomi-chan JVS)
#@markdown You can download this dataset if you don't have your own dataset.
#@markdown Make sure you agree to the license when using this dataset.
#@markdown https://tyc.rei-yumesaki.net/material/corpus/#toc6
# !wget https://tyc.rei-yumesaki.net/files/sozai-tyc-corpus1.zip
# !unzip sozai-tyc-corpus1.zip
# !mv "/content/つくよみちゃんコーパス Vol.1 声優統計コーパス（JVSコーパス準拠）/おまけ：WAV（+12dB増幅＆高音域削減）/WAV（+12dB増幅＆高音域削減）" "dataset_raw/tsukuyomi"

In [ ]:
#@title Automatic preprocessing Resample to 44100Hz and mono
!svc pre-resample

In [ ]:
#@title Divide filelists and generate config.json
!svc pre-config -t {CONFIG_TYPE}

In [ ]:
#@title Backup configs file
update_config()
!cp -r configs drive/MyDrive/so-vits-svc-fork/
!cp -r filelists drive/MyDrive/so-vits-svc-fork/

In [ ]:
#@title Download configs file
!cp -r drive/MyDrive/so-vits-svc-fork/configs .
!cp -r drive/MyDrive/so-vits-svc-fork/filelists .

In [ ]:
#@title Generate hubert and f0
F0_METHOD = "crepe" #@param ["crepe", "crepe-tiny", "parselmouth", "dio", "harvest"]
FORCE_REBUILD_ON = False #@param {type: "boolean"}
if FORCE_REBUILD_ON:
    !svc pre-hubert -fm {F0_METHOD}
else:
    !svc pre-hubert -fm {F0_METHOD} -nf

In [ ]:
#@title Backup or download dataset hubert and f0
DATASET_NAME = CHARACTER

BACKUP_ON = True #@param {type: "boolean"}
if BACKUP_ON:
    !zip -r dataset.zip dataset
    # !zip -r dataset.wav.zip dataset -i dataset/**/*.wav
    # !zip -r dataset.data.pt.zip dataset -i dataset/**/*.data.pt

    !mkdir -p drive/MyDrive/so-vits-svc-fork/datasets/{DATASET_NAME}/
    !rclone mkdir colab:/so-vits-svc-fork/datasets/{DATASET_NAME}/ --config {RCLONE_CONFIG}

    !rm -f *.{{md5,sha1,sha256}}.sum
    !rclone hashsum MD5 . --output-file checksum.md5.sum --filter "+ dataset{{,.wav,.data.pt}}.zip" --filter "- dataset/**" --filter "- *.{{md5,sha1,sha256}}.sum" --filter "- **"
    !rclone hashsum SHA1 . --output-file checksum.sha1.sum --filter "+ dataset{{,.wav,.data.pt}}.zip" --filter "- dataset/**" --filter "- *.{{md5,sha1,sha256}}.sum" --filter "- **"
    !rclone hashsum SHA256 . --output-file checksum.sha256.sum --filter "+ dataset{{,.wav,.data.pt}}.zip" --filter "- dataset/**" --filter "- *.{{md5,sha1,sha256}}.sum" --filter "- **"
    
    # !cp -f dataset{{,.wav,.data.pt}}.zip drive/MyDrive/so-vits-svc-fork/datasets/{DATASET_NAME}/
    !cp -f *.{{md5,sha1,sha256}}.sum drive/MyDrive/so-vits-svc-fork/datasets/{DATASET_NAME}/
    !rclone copy . colab:/so-vits-svc-fork/datasets/{DATASET_NAME}/ --config {RCLONE_CONFIG} --no-traverse --filter "+ dataset{{,.wav,.data.pt}}.zip" --filter "- dataset/**" --filter "- *.{{md5,sha1,sha256}}.sum" --filter "- **"
    !rclone copy . colab:/so-vits-svc-fork/datasets/{DATASET_NAME}/ --config {RCLONE_CONFIG} --no-traverse --filter "+ /*.{{md5,sha1,sha256}}.sum" --filter "- **"

    # !rclone check . drive/MyDrive/so-vits-svc-fork/datasets/{DATASET_NAME}/ --one-way --download --differ - --missing-on-dst - --error - --filter "+ dataset{{,.wav,.data.pt}}.zip" --filter "- dataset/**" --filter "- *.{{md5,sha1,sha256}}.sum" --filter "- **"
    !rclone check . drive/MyDrive/so-vits-svc-fork/datasets/{DATASET_NAME}/ --one-way --download --differ - --missing-on-dst - --error - --filter "+ /*.{{md5,sha1,sha256}}.sum" --filter "- **"
    # !rclone check . colab:/so-vits-svc-fork/datasets/{DATASET_NAME}/ --config {RCLONE_CONFIG} --one-way --download --differ - --missing-on-dst - --error - --filter "+ dataset{{,.wav,.data.pt}}.zip" --filter "- dataset/**" --filter "- *.{{md5,sha1,sha256}}.sum" --filter "- **"
    !rclone check checksum.md5.sum colab:/so-vits-svc-fork/datasets/{DATASET_NAME}/ --config {RCLONE_CONFIG} --checkfile MD5 --one-way --download --differ - --missing-on-dst - --error - --filter "+ dataset{{,.wav,.data.pt}}.zip" --filter "- dataset/**" --filter "- *.{{md5,sha1,sha256}}.sum" --filter "- **"
    !rclone check . colab:/so-vits-svc-fork/datasets/{DATASET_NAME}/ --config {RCLONE_CONFIG} --one-way --download --differ - --missing-on-dst - --error - --filter "+ /*.{{md5,sha1,sha256}}.sum" --filter "- **"

    # !rclone checksum MD5 checksum.md5.sum colab:/so-vits-svc-fork/datasets/{DATASET_NAME}/ --config {RCLONE_CONFIG} --one-way --download --differ - --missing-on-dst - --error - --filter "+ dataset{{,.wav,.data.pt}}.zip" --filter "- dataset/**" --filter "- *.{{md5,sha1,sha256}}.sum" --filter "- **"

    # !ps -ef | grep rclone
    # !iftop -t -s 1 -n
else:
    !cp -f drive/MyDrive/so-vits-svc-fork/datasets/{DATASET_NAME}/*.{{md5,sha1,sha256}}.sum .
    !rclone copy colab:/so-vits-svc-fork/datasets/{DATASET_NAME}/ . --config {RCLONE_CONFIG} --no-traverse --filter "+ dataset{{,.wav,.data.pt}}.zip" --filter "- dataset/**" --filter "- *.{{md5,sha1,sha256}}.sum" --filter "- **"

    !rclone check checksum.md5.sum . --checkfile MD5 --one-way --differ - --missing-on-dst - --error - --filter "+ dataset{{,.wav,.data.pt}}.zip" --filter "- dataset/**" --filter "- *.{{md5,sha1,sha256}}.sum" --filter "- **"
    # !rclone checksum MD5 checksum.md5.sum . --one-way --differ - --missing-on-dst - --error - --filter "+ dataset{{,.wav,.data.pt}}.zip" --filter "- dataset/**" --filter "- *.{{md5,sha1,sha256}}.sum" --filter "- **"

    # !unzip -d ./ drive/MyDrive/so-vits-svc-fork/datasets/{DATASET_NAME}/dataset.zip
    !unzip -d ./ dataset.zip

In [ ]:
#@title Check dataset
!ls dataset/*/*/* | wc -l
!ls dataset/*/*/*.wav | wc -l
!ls dataset/*/*/*.data.pt | wc -l
!ls -lt dataset/**/* | tail -n 10

## Training

In [ ]:
import os
from logging import getLogger
from pathlib import Path

import lightning.pytorch as pl
import torch
from lightning.pytorch.loggers import TensorBoardLogger
from lightning.pytorch.strategies.ddp import DDPStrategy
from lightning.pytorch.tuner import Tuner

import so_vits_svc_fork.utils

from so_vits_svc_fork import utils
from so_vits_svc_fork.logger import is_notebook

from so_vits_svc_fork.train import VitsLightning, VCDataModule

LOG = getLogger(__name__)
torch.set_float32_matmul_precision("high")


def train(
    config_path: Path | str, model_path: Path | str, reset_optimizer: bool = False
):
    config_path = Path(config_path)
    model_path = Path(model_path)

    hparams = utils.get_backup_hparams(config_path, model_path)
    utils.ensure_pretrained_model(model_path, hparams.model.get("type_", "hifi-gan"))

    datamodule = VCDataModule(hparams)
    strategy = (
        (
            "ddp_find_unused_parameters_true"
            if os.name != "nt"
            else DDPStrategy(find_unused_parameters=True, process_group_backend="gloo")
        )
        if torch.cuda.device_count() > 1
        else "auto"
    )
    LOG.info(f"Using strategy: {strategy}")
    trainer = pl.Trainer(
        logger=TensorBoardLogger(
            model_path, "lightning_logs", hparams.train.get("log_version", 0)
        ),
        # profiler="simple",
        val_check_interval=hparams.train.eval_interval,
        max_epochs=hparams.train.epochs,
        check_val_every_n_epoch=None,
        precision="16-mixed"
        if hparams.train.fp16_run
        else "bf16-mixed"
        if hparams.train.get("bf16_run", False)
        else 32,
        strategy=strategy,
        callbacks=[pl.callbacks.RichProgressBar()] if not is_notebook() else None,
        benchmark=True,
        enable_checkpointing=False,
    )
    tuner = Tuner(trainer)
    model = VitsLightning(reset_optimizer=reset_optimizer, **hparams)

    # automatic batch size scaling
    batch_size = hparams.train.batch_size
    batch_split = str(batch_size).split("-")
    batch_size = batch_split[0]
    init_val = 2 if len(batch_split) <= 1 else int(batch_split[1])
    max_trials = 25 if len(batch_split) <= 2 else int(batch_split[2])
    if batch_size == "auto":
        batch_size = "binsearch"
    if batch_size in ["power", "binsearch"]:
        model.tuning = True
        tuner.scale_batch_size(
            model,
            mode=batch_size,
            datamodule=datamodule,
            steps_per_trial=1,
            init_val=init_val,
            max_trials=max_trials,
        )
        model.tuning = False
    else:
        batch_size = int(batch_size)
    # automatic learning rate scaling is not supported for multiple optimizers
    """if hparams.train.learning_rate  == "auto":
    lr_finder = tuner.lr_find(model)
    LOG.info(lr_finder.results)
    fig = lr_finder.plot(suggest=True)
    fig.savefig(model_path / "lr_finder.png")"""

    trainer.fit(model, datamodule=datamodule)

if __name__ == '__main__':
    # train(CONFIG_FILE, MODEL_PATH)

In [ ]:
#@title Train
%load_ext tensorboard
%tensorboard --logdir {MODEL_PATH}
# !svc train --model-path {MODEL_PATH}
train(CONFIG_FILE, MODEL_PATH)

## Training Cluster model

In [ ]:
#@title Train cluster model (Optional)
!svc train-cluster --output-path {MODEL_PATH}kmeans.pt

## Inference

In [ ]:
#@title Get the author's voice as a source
import random
NAME = str(random.randint(1, 49))
TYPE = "fsd50k" #@param ["", "digit", "dog", "fsd50k"]
CUSTOM_FILEPATH = "" #@param {type: "string"}
if CUSTOM_FILEPATH != "":
    NAME = CUSTOM_FILEPATH
else:
    # it is extremely difficult to find a voice that can download from the internet directly
    if TYPE == "dog":
        !wget -N f"https://huggingface.co/datasets/437aewuh/dog-dataset/resolve/main/dogs/dogs_{NAME:.0000}.wav" -O {NAME}.wav
    elif TYPE == "digit":
        # george, jackson, lucas, nicolas, ...
        !wget -N f"https://github.com/Jakobovski/free-spoken-digit-dataset/raw/master/recordings/0_george_{NAME}.wav" -O {NAME}.wav
    elif TYPE == "fsd50k":
        !wget -N f"https://huggingface.co/datasets/Fhrozen/FSD50k/blob/main/clips/dev/{10000+int(NAME)}.wav" -O {NAME}.wav
    else:
        !wget -N f"https://zunko.jp/sozai/utau/voice_{"kiritan" if NAME < 25 else "itako"}{NAME % 5 + 1}.wav" -O {NAME}.wav

from IPython.display import Audio, display

display(Audio(f"{NAME}.wav"))

In [ ]:
#@title Use trained model
#@markdown **Put your .wav file in `so-vits-svc-fork/audio` directory**
from IPython.display import Audio, display

AUDIO_PATH = 'drive/MyDrive/so-vits-svc-fork/audio/' #@param {type: "string"}
NAME = "test" #@param {type: "string"}
TRANSPOSE = 0 #@param {type: "number"}
F0_METHOD = "crepe" #@param ["crepe", "crepe-tiny", "parselmouth", "dio", "harvest"]

AUTO_PREDICT_F0 = True #@param {type:"boolean"}
if AUTO_PREDICT_F0:
    !svc infer {AUDIO_PATH}{NAME}.wav -m {MODEL_PATH} -c {MODEL_PATH}config.json -t {TRANSPOSE} -fm {F0_METHOD}
else:
    !svc infer {AUDIO_PATH}{NAME}.wav -m {MODEL_PATH} -c {MODEL_PATH}config.json -t {TRANSPOSE} -fm {F0_METHOD} -na

display(Audio(f"{AUDIO_PATH}{NAME}.wav", autoplay=False))
display(Audio(f"{AUDIO_PATH}{NAME}.out.wav", autoplay=False))

In [ ]:
##@title Use trained model (with cluster)
from IPython.display import Audio, display

AUDIO_PATH = 'drive/MyDrive/so-vits-svc-fork/audio/' #@param {type: "string"}
NAME = "test" #@param {type: "string"}
SPEAKER = "kiritan" #@param {type: "string"}
TRANSPOSE = 0 #@param {type: "number"}
F0_METHOD = "crepe" #@param ["crepe", "crepe-tiny", "parselmouth", "dio", "harvest"]

AUTO_PREDICT_F0 = True #@param {type:"boolean"}
if AUTO_PREDICT_F0:
    !svc infer {AUDIO_PATH}{NAME}.wav -s {SPEAKER} -r 0.1 -m {MODEL_PATH} -c {MODEL_PATH}config.json -k {MODEL_PATH}kmeans.pt -t {TRANSPOSE} -fm {F0_METHOD}
else:
    !svc infer {AUDIO_PATH}{NAME}.wav -s {SPEAKER} -r 0.1 -m {MODEL_PATH} -c {MODEL_PATH}config.json -k {MODEL_PATH}kmeans.pt -t {TRANSPOSE} -fm {F0_METHOD} -na

display(Audio(f"{AUDIO_PATH}{NAME}.wav", autoplay=False))
display(Audio(f"{AUDIO_PATH}{NAME}.out.wav", autoplay=False))

### Backup models

In [ ]:
DATASET_NAME = CHARACTER

!rm -f *.{{md5,sha1,sha256}}.sum
!rclone mkdir colab:/so-vits-svc-fork/models/{DATASET_NAME}/ --config {RCLONE_CONFIG}
!rclone copy colab:/so-vits-svc-fork/models/{DATASET_NAME}/ . --config {RCLONE_CONFIG} --no-traverse --filter "+ /*.{{md5,sha1,sha256}}.sum" --filter "- **"

# !rclone ls colab:/so-vits-svc-fork/models/{DATASET_NAME}/ --config {RCLONE_CONFIG} --filter "+ /*.{{md5,sha1,sha256}}.sum" --filter "- **"
# !rclone ls . --filter "+ /*.{{md5,sha1,sha256}}.sum" --filter "- **"
# !rclone ls drive/MyDrive/so-vits-svc-fork/logs/44k/ --filter "+ config.json" --filter "- {{D,G}}_0.pth" --filter "+ {{D,G}}_\d\d\d*.pth" --filter "+ kmeans.pt" --filter "- *.{{md5,sha1,sha256}}.sum" --filter "- **"

!sort -k 2.3bn -k 2b -k 1 -u <(rclone hashsum MD5 drive/MyDrive/so-vits-svc-fork/logs/44k/ --filter "+ config.json" --filter "- {{D,G}}_0.pth" --filter "+ {{D,G}}_\d\d\d*.pth" --filter "+ kmeans.pt" --filter "- *.{{md5,sha1,sha256}}.sum" --filter "- **") <(cat checksum.md5.sum) -o checksum.md5.sum
!sort -k 2.3bn -k 2b -k 1 -u <(rclone hashsum SHA1 drive/MyDrive/so-vits-svc-fork/logs/44k/ --filter "+ config.json" --filter "- {{D,G}}_0.pth" --filter "+ {{D,G}}_\d\d\d*.pth" --filter "+ kmeans.pt" --filter "- *.{{md5,sha1,sha256}}.sum" --filter "- **") <(cat checksum.sha1.sum) -o checksum.sha1.sum
!sort -k 2.3bn -k 2b -k 1 -u <(rclone hashsum SHA256 drive/MyDrive/so-vits-svc-fork/logs/44k/ --filter "+ config.json" --filter "- {{D,G}}_0.pth" --filter "+ {{D,G}}_\d\d\d*.pth" --filter "+ kmeans.pt" --filter "- *.{{md5,sha1,sha256}}.sum" --filter "- **") <(cat checksum.sha256.sum) -o checksum.sha256.sum

!cp -f *.{{md5,sha1,sha256}}.sum drive/MyDrive/so-vits-svc-fork/logs/44k/
!rclone copy drive/MyDrive/so-vits-svc-fork/logs/44k/ colab:/so-vits-svc-fork/models/{DATASET_NAME}/ --config {RCLONE_CONFIG} --no-traverse --filter "+ config.json" --filter "- {{D,G}}_0.pth" --filter "+ {{D,G}}_\d\d\d*.pth" --filter "+ kmeans.pt" --filter "- *.{{md5,sha1,sha256}}.sum" --filter "- **"
!rclone copy . colab:/so-vits-svc-fork/models/{DATASET_NAME}/ --config {RCLONE_CONFIG} --no-traverse --filter "+ /*.{{md5,sha1,sha256}}.sum" --filter "- **"

!rclone check . drive/MyDrive/so-vits-svc-fork/logs/44k/ --one-way --download --differ - --missing-on-dst - --error - --filter "+ /*.{{md5,sha1,sha256}}.sum" --filter "- **"
!rclone check drive/MyDrive/so-vits-svc-fork/logs/44k/ colab:/so-vits-svc-fork/models/{DATASET_NAME}/ --config {RCLONE_CONFIG} --one-way --download --differ - --missing-on-dst - --error - --filter "+ config.json" --filter "- {{D,G}}_0.pth" --filter "+ {{D,G}}_\d\d\d*.pth" --filter "+ kmeans.pt" --filter "- *.{{md5,sha1,sha256}}.sum" --filter "- **"
!rclone check . colab:/so-vits-svc-fork/models/{DATASET_NAME}/ --config {RCLONE_CONFIG} --one-way --download --differ - --missing-on-dst - --error - --filter "+ /*.{{md5,sha1,sha256}}.sum" --filter "- **"

### Pretrained models

In [ ]:
#@title https://huggingface.co/TachibanaKimika/so-vits-svc-4.0-models/tree/main
!wget -N "https://huggingface.co/TachibanaKimika/so-vits-svc-4.0-models/resolve/main/riri/G_riri_220.pth"
!wget -N "https://huggingface.co/TachibanaKimika/so-vits-svc-4.0-models/resolve/main/riri/config.json"

In [ ]:
from IPython.display import Audio, display

AUDIO_PATH = 'drive/MyDrive/so-vits-svc-fork/audio/' #@param {type: "string"}
NAME = "test" #@param {type: "string"}

AUTO_PREDICT_F0 = True #@param {type:"boolean"}
if AUTO_PREDICT_F0:
    !svc infer {AUDIO_PATH}{NAME}.wav -c config.json -m G_riri_220.pth
else:
    !svc infer {AUDIO_PATH}{NAME}.wav -c config.json -m G_riri_220.pth -na
display(Audio(f"{AUDIO_PATH}{NAME}.wav", autoplay=False))
display(Audio(f"{AUDIO_PATH}{NAME}.out.wav", autoplay=False))

In [ ]:
#@title https://huggingface.co/therealvul/so-vits-svc-4.0/tree/main
!wget -N "https://huggingface.co/therealvul/so-vits-svc-4.0/resolve/main/Pinkie%20(speaking%20sep)/G_166400.pth"
!wget -N "https://huggingface.co/therealvul/so-vits-svc-4.0/resolve/main/Pinkie%20(speaking%20sep)/config.json"

In [ ]:
from IPython.display import Audio, display

AUDIO_PATH = 'drive/MyDrive/so-vits-svc-fork/audio/' #@param {type: "string"}
NAME = "test" #@param {type: "string"}

AUTO_PREDICT_F0 = True #@param {type:"boolean"}
if AUTO_PREDICT_F0:
    !svc infer {AUDIO_PATH}{NAME}.wav --speaker "Pinkie {neutral}" -c config.json -m G_166400.pth
else:
    !svc infer {AUDIO_PATH}{NAME}.wav --speaker "Pinkie {neutral}" -c config.json -m G_166400.pth -na
display(Audio(f"{AUDIO_PATH}{NAME}.wav", autoplay=False))
display(Audio(f"{AUDIO_PATH}{NAME}.out.wav", autoplay=False))